In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader

# ResNet50 모델 불러오기
model = models.resnet50(pretrained=True)

# 마지막 fully connected layer 수정
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1),  # 1개의 출력 (0- 1)
    nn.Sigmoid()             # Sigmoid 활성화 함수 추가
)

# 데이터셋과 데이터 로더 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet50은 기본적으로 224x224 이미지를 입력으로 사용
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss

# 옵티마이저와 손실 함수 설정
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = FocalLoss()

# CUDA 사용 설정 (GPU 사용 시)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


C:\Users\lwj01\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\lwj01\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
import os
import pandas as pd
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CustomDataset 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, dir_path_ball, dir_path_background, transform=None):
        self.dir_path_ball = dir_path_ball
        self.dir_path_background = dir_path_background
        self.transform = transform
        
        # 이미지 파일 경로와 라벨을 리스트로 만듭니다.
        self.image_paths = []
        self.labels = []

        # 라벨이 1인 이미지 (테니스공)
        for filename in os.listdir(self.dir_path_ball):
            if filename.endswith('.png'):  # .png 파일만 추가
                self.image_paths.append(os.path.join(self.dir_path_ball, filename))
                self.labels.append(1)
        
        # 라벨이 0인 이미지 (배경)
        for filename in os.listdir(self.dir_path_background):
            if filename.endswith('.png'):  # .png 파일만 추가
                self.image_paths.append(os.path.join(self.dir_path_background, filename))
                self.labels.append(0)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        # 이미지를 로드하고 라벨을 가져옵니다.
        image = Image.open(self.image_paths[index])
        label = self.labels[index]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


dir_path_ball = r'C:\Users\lwj01\HowFastTennisBallIs\novak_sinner_over_30\cropped_ball\augmentation'
dir_path_background = r'C:\Users\lwj01\HowFastTennisBallIs\novak_sinner_over_30\cropped_baseground'

train_dataset = CustomDataset(
    dir_path_ball=dir_path_ball,
    dir_path_background=dir_path_background,
    transform=transform
)

train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)



# 학습 과정
num_epochs = 1
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = torch.squeeze(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete.")

OutOfMemoryError: CUDA out of memory. Tried to allocate 98.00 MiB. GPU 0 has a total capacty of 6.00 GiB of which 0 bytes is free. Of the allocated memory 19.56 GiB is allocated by PyTorch, and 136.15 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [3]:
torch.save(model, 'model_v4.pt')